# Test the functionality of DataPlane Python bindings

Mimic the `e2sar_reas_test` in the C++ unit test suite.

In [2]:
import time
import sys

## IMPORTANT: Update the path to your built Python module. Use the absolute path to make life easier.
sys.path.append(
    '/home/xmei/Documents/hpdf_projects/E2SAR/build/src/pybind')

import e2sar_py

In [ ]:

dir(e2sar_py.DataPlane)

## DPReasTest1

This is a test that uses local host to send/receive fragments. It does NOT use control plane.


1. First confirm the "Reassembler" bindings are working.

In [ ]:
# Set the reassembler URI
REAS_URI_ = "ejfat://useless@192.168.100.1:9876/lb/1?sync=192.168.0.1:12345&data=127.0.0.1"
reas_uri = e2sar_py.EjfatURI(uri=REAS_URI_, tt=e2sar_py.EjfatURI.TokenType.instance)

# Make sure the token matches the one in the string
def get_inst_token(uri : e2sar_py.EjfatURI):
    try:
        token = uri.get_instance_token().value()
        print("Instance Token:", token)
    except RuntimeError as e:
        print("Instance Token - Error:", e)

get_inst_token(reas_uri)

# Get data plane IPv4 address
assert(str(reas_uri.get_data_addr_v4().value()[0]) == "127.0.0.1")
print("Data Plane Address (v4):", str(reas_uri.get_data_addr_v4().value()[0]))

In [ ]:
# Config the reassembler flags

rflags = e2sar_py.DataPlane.ReassemblerFlags()
rflags.useCP = False  # turn off CP. Default value is True
rflags.withLBHeader = True  # LB header will be attached since there is no LB

assert rflags.period_ms == 100  # default value of the C++ constructor
assert rflags.useCP == False

print("Reassembler flags:")
print(f"  period_ms={rflags.period_ms}")  # should be 100 according to the C++ constructor
print(f"  useCP={rflags.useCP}")
print(f"  validateCert = {rflags.validateCert}")
print(f"  epoch_ms = {rflags.epoch_ms}")
print(f"  setPoint = {rflags.setPoint}")
print(f"  [Kp, Ki, Kd] = [{rflags.Kp}, {rflags.Ki}, {rflags.Kd}]")
print(f"  dpV6 = {rflags.dpV6}")
print(f"  cpV6 = {rflags.cpV6}")
print(f"  portRange = {rflags.portRange}")
print(f"  withLBHeader = {rflags.withLBHeader}")


In [ ]:
dir(e2sar_py.DataPlane.Reassembler)

In [ ]:
# Init the reassembler object
# print(type(reas_uri), type(1), type(rflags))
reas = e2sar_py.DataPlane.Reassembler(reas_uri, 1, rflags)

2. The "Segmenter" bindings.

In [3]:
# Set up URI for segmenter
SEG_URI = "ejfat://useless@192.168.100.1:9876/lb/1?sync=192.168.0.1:12345&data=127.0.0.1"
seg_uri = e2sar_py.EjfatURI(uri=SEG_URI, tt=e2sar_py.EjfatURI.TokenType.instance)

# Set up sflags
sflags = e2sar_py.DataPlane.SegmenterFlags()
sflags.useCP = False  # turn off CP. Default value is True
sflags.syncPeriodMs = 1000
sflags.syncPeriods = 5

assert(sflags.syncPeriodMs == 1000)
assert(sflags.useCP == False)
assert(sflags.syncPeriods == 5)

print("Segmenter flags:")
print(f"  syncPeriodMs={sflags.syncPeriodMs}")
print(f"  useCP={sflags.useCP}")
print(f"  mtu={sflags.mtu}")
print(f"  syncPeriods={sflags.syncPeriods}")

Segmenter flags:
  syncPeriodMs=1000
  useCP=False
  mtu=1500
  syncPeriods=5


In [4]:
# Init segmenter object
data_id = 0x0505
eventSrc_id = 0x11223344

seg = e2sar_py.DataPlane.Segmenter(seg_uri, data_id, eventSrc_id, sflags)

In [5]:
# Start segmenter threads
res = seg.OpenAndStart()

try:
    assert res.value() == 0
except RuntimeError as e:
    print("Segmenter::OpenAndStart error:", e)

res = seg.getSendStats()
try:
    assert(res[1] == 0)
except RuntimeError as e:
    print(f"Error encountered after opening send socket: {res[2]}")

In [6]:
SEND_STR = "THIS IS A VERY LONG EVENT MESSAGE WE WANT TO SEND EVERY 1 SECONDS."

send_context = SEND_STR.encode('utf-8')
print(send_context)
print(len(send_context))

b'THIS IS A VERY LONG EVENT MESSAGE WE WANT TO SEND EVERY 1 SECONDS.'
66


In [7]:
# Send the data
for i in range(5):
    print(f"Round {i + 1}")
    res = seg.addToSendQueue(send_context, len(send_context))
    assert(res.value() == 0)
    res = seg.getSendStats()
    # print(res)
    if (res[1] != 0):
        print(f"  Error encountered sending event frame: {i}, errormsg: {res[2]}")
    time.sleep(1)

Round 1
Round 2
Round 3
Round 4
Round 5


In [8]:
res = seg.getSendStats()
assert(res[0] == 5)
print(f"Sent {res[0]} data frames")

if (res[1] != 0):
    print(f"Error --  code: {res[1]}; errormsg: {res[2]}")


Sent 5 data frames
Error --  code: 1; errormsg: 111


## DPReasTest2

Test segmentation and reassembly on local host with no control plane (basic segmentation) using small MTU.

In [9]:
# Set up sflags
sflags = e2sar_py.DataPlane.SegmenterFlags()
sflags.useCP = False  # turn off CP. Default value is True
sflags.syncPeriodMs = 1000
sflags.syncPeriods = 5
sflags.mtu = 80

assert(sflags.syncPeriodMs == 1000)
assert(sflags.useCP == False)
assert(sflags.syncPeriods == 5)
assert(sflags.mtu == 80)

print("Segmenter flags:")
print(f"  syncPeriodMs={sflags.syncPeriodMs}")
print(f"  useCP={sflags.useCP}")
print(f"  mtu={sflags.mtu}")
print(f"  syncPeriods={sflags.syncPeriods}")

Segmenter flags:
  syncPeriodMs=1000
  useCP=False
  mtu=80
  syncPeriods=5


In [10]:
seg = e2sar_py.DataPlane.Segmenter(seg_uri, data_id, eventSrc_id, sflags)

res = seg.OpenAndStart()

try:
    assert(res.value() == 0)
except RuntimeError as e:
    print("Segmenter::OpenAndStart error:", e)

res = seg.getSendStats()
try:
    assert(res[1] == 0)
except RuntimeError as e:
    print(f"Error encountered after opening send socket: {res[2]}")



In [12]:
for i in range(5):
    print(f"Round {i + 1}")
    res = seg.addToSendQueue(send_context, len(send_context))
    assert(res.value() == 0)
    res = seg.getSendStats()
    if (res[1] != 0):
        print(f"  SendStats: {res}")
        print(f"  Error encountered sending event frame: {i}, errormsg: {res[2]}")
    time.sleep(5)

Round 1
  SendStats: (10, 5, 111)
  Error encountered sending event frame: 0, errormsg: 111
Round 2
  SendStats: (13, 6, 111)
  Error encountered sending event frame: 1, errormsg: 111
Round 3
  SendStats: (14, 7, 111)
  Error encountered sending event frame: 2, errormsg: 111
Round 4
  SendStats: (17, 8, 111)
  Error encountered sending event frame: 3, errormsg: 111
Round 5
  SendStats: (18, 9, 111)
  Error encountered sending event frame: 4, errormsg: 111
